In [2]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta, time
from IPython.display import display

# for jupyt notebooks
import os
d = os.path.dirname(os.path.dirname(os.getcwd()))
print(d)
import sys
sys.path.append(d)
from ATJ.mt5_trade_utils.backtester import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

c:\Users\derne\OneDrive - The Pennsylvania State University\Programming\Extra\algobot


In [4]:
symbol = 'USDJPYm'
start_dt = datetime(2023, 1, 1)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc = get_ohlc_history(symbol, mt5.TIMEFRAME_D1, start_dt, end_dt)

sma_period = 20

ohlc['sma_20'] = ohlc['open'].rolling(sma_period).mean()
ohlc['std'] = ohlc['open'].rolling(sma_period).std()
ohlc['upper_band'] = ohlc['sma_20'] + 1.5 * ohlc['std']
ohlc['lower_band'] = ohlc['sma_20'] - 1.5 * ohlc['std']


display(ohlc)

fig = create_price_fig(ohlc, indicators=['sma_20', 'upper_band', 'lower_band'])
display(fig)

,time,open,high,low,close,sma_20,std,upper_band,lower_band
0,2023-01-02,130.877,131.334,130.397,130.694,NaN,NaN,NaN,NaN
1,2023-01-03,130.749,131.416,129.498,131.291,NaN,NaN,NaN,NaN
2,2023-01-04,131.293,132.715,129.923,132.242,NaN,NaN,NaN,NaN
3,2023-01-05,132.244,134.050,131.679,133.317,NaN,NaN,NaN,NaN
4,2023-01-06,133.319,134.770,131.992,132.095,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
640,2025-01-20,156.307,156.467,155.160,155.268,157.32765,0.777605,158.494057,156.161243
641,2025-01-21,155.270,156.235,154.770,155.741,157.20680,0.897406,158.552909,155.860691
642,2025-01-22,155.744,156.704,155.350,156.416,157.09905,0.938359,158.506589,155.691511
643,2025-01-23,156.432,156.749,155.738,156.135,157.07030,0.950064,158.495396,155.645204


Bollinger Band strategy that:

- Buys when the price breaks below the lower band

- Sells when the price breaks above the upper band
- Closes buy positions when the price moves above the 20-period SMA
- Closes sell positions when the price moves below the lower band

In [ ]:
def get_signal(x):
    if x['open'] > x['upper_band']:
        return 'sell'
    elif x['open'] < x['lower_band']:
        return 'buy'
    
def get_exit_signal(x):
    if x['open'] > x['sma_20']:
        return 'close buy positions'
    elif x['open'] < x['lower_band']:
        return 'close sell positions'
    
ohlc['signal'] = ohlc.apply(get_signal, axis=1)
ohlc['exit_signal'] = ohlc.apply(get_exit_signal, axis=1)
ohlc

,time,open,high,low,close,sma_20,std,upper_band,lower_band,signal,exit_signal
0,2023-01-02,130.877,131.334,130.397,130.694,NaN,NaN,NaN,NaN,None,None
1,2023-01-03,130.749,131.416,129.498,131.291,NaN,NaN,NaN,NaN,None,None
2,2023-01-04,131.293,132.715,129.923,132.242,NaN,NaN,NaN,NaN,None,None
3,2023-01-05,132.244,134.050,131.679,133.317,NaN,NaN,NaN,NaN,None,None
4,2023-01-06,133.319,134.770,131.992,132.095,NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...
640,2025-01-20,156.307,156.467,155.160,155.268,157.32765,0.777605,158.494057,156.161243,None,None
641,2025-01-21,155.270,156.235,154.770,155.741,157.20680,0.897406,158.552909,155.860691,buy,close sell positions
642,2025-01-22,155.744,156.704,155.350,156.416,157.09905,0.938359,158.506589,155.691511,None,None
643,2025-01-23,156.432,156.749,155.738,156.135,157.07030,0.950064,158.495396,155.645204,None,None


### create trade logic

- Enters a new trade when the Bollinger Band strategy generates a buy or sell signal and there are no open trades.
- Exits an existing trade when the Bollinger Band strategy generates an exit signal and the trade is in the correct direction (buy or sell).

In [6]:
# create trade logic
def on_bar(data, trades, orders):
    volume = 100000 # 1 lots
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        orders.open_trade(symbol, volume, 'sell')
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data['exit_signal'] == 'close buy positions':
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and data['exit_signal'] == 'close sell positions':
            orders.close_trade(trade)

In [8]:
# backtest parameters
starting_balance = 100 
currency = 'USD'
exchange_rate = 1/150
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,USDJPYm,sell,100000,2023-02-05 00:00:00,132.182,2023-03-14 00:00:00,133.087,0,0,{},-603.33,-7.0,-610.33,-610.33,-510.33
1,closed,USDJPYm,buy,100000,2023-03-16 00:00:00,132.901,2023-03-31 00:00:00,133.198,0,0,{},198.00,-7.0,191.0,-419.33,-319.33
2,closed,USDJPYm,sell,100000,2023-04-11 00:00:00,133.464,2023-07-11 00:00:00,141.305,0,0,{},-5227.33,-7.0,-5234.33,-5653.66,-5553.66
3,closed,USDJPYm,buy,100000,2023-07-12 00:00:00,140.116,2023-07-23 00:00:00,141.655,0,0,{},1026.00,-7.0,1019.0,-4634.66,-4534.66
4,closed,USDJPYm,sell,100000,2023-08-01 00:00:00,142.323,2023-11-21 00:00:00,148.381,0,0,{},-4038.67,-7.0,-4045.67,-8680.33,-8580.33
5,closed,USDJPYm,buy,100000,2023-11-22 00:00:00,148.146,2024-01-04 00:00:00,142.994,0,0,{},-3434.67,-7.0,-3441.67,-12122.0,-12022.0
6,closed,USDJPYm,sell,100000,2024-01-08 00:00:00,144.718,2024-03-07 00:00:00,149.261,0,0,{},-3028.67,-7.0,-3035.67,-15157.67,-15057.67
7,closed,USDJPYm,buy,100000,2024-03-08 00:00:00,147.861,2024-03-19 00:00:00,149.149,0,0,{},858.67,-7.0,851.67,-14306.0,-14206.0
8,closed,USDJPYm,sell,100000,2024-03-22 00:00:00,151.626,2024-06-05 00:00:00,155.113,0,0,{},-2324.67,-7.0,-2331.67,-16637.67,-16537.67
9,closed,USDJPYm,sell,100000,2024-06-21 00:00:00,158.931,2024-07-12 00:00:00,158.279,0,0,{},434.67,-7.0,427.67,-16210.0,-16110.0


In [9]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [10]:
backtest_fig = bt.visualize_backtest(indicators=['sma_20', 'upper_band', 'lower_band'])
backtest_fig

In [11]:
evaluate_backtest(bt.trades)

biggest_profit 1475.33
biggest_loss -5830.67


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
1,closed,USDJPYm,buy,100000,2023-03-16,132.901,2023-03-31,133.198,0,0,{},198.00,-7.0,191.0,-419.33,-319.33
3,closed,USDJPYm,buy,100000,2023-07-12,140.116,2023-07-23,141.655,0,0,{},1026.00,-7.0,1019.0,-4634.66,-4534.66
7,closed,USDJPYm,buy,100000,2024-03-08,147.861,2024-03-19,149.149,0,0,{},858.67,-7.0,851.67,-14306.0,-14206.0
9,closed,USDJPYm,sell,100000,2024-06-21,158.931,2024-07-12,158.279,0,0,{},434.67,-7.0,427.67,-16210.0,-16110.0
11,closed,USDJPYm,buy,100000,2024-08-25,144.185,2024-09-02,146.398,0,0,{},1475.33,-7.0,1468.33,-20579.34,-20479.34
12,closed,USDJPYm,buy,100000,2024-09-05,143.33,2024-09-22,143.82,0,0,{},326.67,-7.0,319.67,-20259.67,-20159.67
14,closed,USDJPYm,buy,100000,2024-11-29,151.181,2024-12-12,152.257,0,0,{},717.33,-7.0,710.33,-23978.34,-23878.34
16,closed,USDJPYm,buy,100000,2025-01-17,155.363,2025-01-24,155.984,0,0,{},414.00,-7.0,407.0,-24617.01,-24517.01


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,USDJPYm,sell,100000,2023-02-05,132.182,2023-03-14,133.087,0,0,{},-603.33,-7.0,-610.33,-610.33,-510.33
2,closed,USDJPYm,sell,100000,2023-04-11,133.464,2023-07-11,141.305,0,0,{},-5227.33,-7.0,-5234.33,-5653.66,-5553.66
4,closed,USDJPYm,sell,100000,2023-08-01,142.323,2023-11-21,148.381,0,0,{},-4038.67,-7.0,-4045.67,-8680.33,-8580.33
5,closed,USDJPYm,buy,100000,2023-11-22,148.146,2024-01-04,142.994,0,0,{},-3434.67,-7.0,-3441.67,-12122.0,-12022.0
6,closed,USDJPYm,sell,100000,2024-01-08,144.718,2024-03-07,149.261,0,0,{},-3028.67,-7.0,-3035.67,-15157.67,-15057.67
8,closed,USDJPYm,sell,100000,2024-03-22,151.626,2024-06-05,155.113,0,0,{},-2324.67,-7.0,-2331.67,-16637.67,-16537.67
10,closed,USDJPYm,buy,100000,2024-07-14,157.974,2024-08-16,149.228,0,0,{},-5830.67,-7.0,-5837.67,-22047.67,-21947.67
13,closed,USDJPYm,sell,100000,2024-09-26,144.707,2024-11-28,151.34,0,0,{},-4422.00,-7.0,-4429.0,-24688.67,-24588.67
15,closed,USDJPYm,sell,100000,2024-12-19,154.661,2025-01-16,156.219,0,0,{},-1038.67,-7.0,-1045.67,-25024.01,-24924.01


avg_win 681.33375
avg_loss -3327.6311111111113
count_profit_trades 8
count_loss_trades 9
win_rate 0.89
rrr 0.2


,order_type,profit
0,buy,-4249.34
1,sell,-20248.67


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,dayofweek
0,closed,USDJPYm,sell,100000,2023-02-05,132.182,2023-03-14,133.087,0,0,{},-603.33,-7.0,-610.33,-610.33,-510.33,1
1,closed,USDJPYm,buy,100000,2023-03-16,132.901,2023-03-31,133.198,0,0,{},198.00,-7.0,191.0,-419.33,-319.33,4
2,closed,USDJPYm,sell,100000,2023-04-11,133.464,2023-07-11,141.305,0,0,{},-5227.33,-7.0,-5234.33,-5653.66,-5553.66,1
3,closed,USDJPYm,buy,100000,2023-07-12,140.116,2023-07-23,141.655,0,0,{},1026.00,-7.0,1019.0,-4634.66,-4534.66,6
4,closed,USDJPYm,sell,100000,2023-08-01,142.323,2023-11-21,148.381,0,0,{},-4038.67,-7.0,-4045.67,-8680.33,-8580.33,1
5,closed,USDJPYm,buy,100000,2023-11-22,148.146,2024-01-04,142.994,0,0,{},-3434.67,-7.0,-3441.67,-12122.0,-12022.0,3
6,closed,USDJPYm,sell,100000,2024-01-08,144.718,2024-03-07,149.261,0,0,{},-3028.67,-7.0,-3035.67,-15157.67,-15057.67,3
7,closed,USDJPYm,buy,100000,2024-03-08,147.861,2024-03-19,149.149,0,0,{},858.67,-7.0,851.67,-14306.0,-14206.0,1
8,closed,USDJPYm,sell,100000,2024-03-22,151.626,2024-06-05,155.113,0,0,{},-2324.67,-7.0,-2331.67,-16637.67,-16537.67,2
9,closed,USDJPYm,sell,100000,2024-06-21,158.931,2024-07-12,158.279,0,0,{},434.67,-7.0,427.67,-16210.0,-16110.0,4


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,dayofweek,current_max,drawdown
0,closed,USDJPYm,sell,100000,2023-02-05,132.182,2023-03-14,133.087,0,0,{},-603.33,-7.0,-610.33,-610.33,-510.33,1,-610.33,0.0
1,closed,USDJPYm,buy,100000,2023-03-16,132.901,2023-03-31,133.198,0,0,{},198.00,-7.0,191.0,-419.33,-319.33,4,-419.33,0.0
2,closed,USDJPYm,sell,100000,2023-04-11,133.464,2023-07-11,141.305,0,0,{},-5227.33,-7.0,-5234.33,-5653.66,-5553.66,1,-419.33,-5234.33
3,closed,USDJPYm,buy,100000,2023-07-12,140.116,2023-07-23,141.655,0,0,{},1026.00,-7.0,1019.0,-4634.66,-4534.66,6,-419.33,-4215.33
4,closed,USDJPYm,sell,100000,2023-08-01,142.323,2023-11-21,148.381,0,0,{},-4038.67,-7.0,-4045.67,-8680.33,-8580.33,1,-419.33,-8261.0
5,closed,USDJPYm,buy,100000,2023-11-22,148.146,2024-01-04,142.994,0,0,{},-3434.67,-7.0,-3441.67,-12122.0,-12022.0,3,-419.33,-11702.67
6,closed,USDJPYm,sell,100000,2024-01-08,144.718,2024-03-07,149.261,0,0,{},-3028.67,-7.0,-3035.67,-15157.67,-15057.67,3,-419.33,-14738.34
7,closed,USDJPYm,buy,100000,2024-03-08,147.861,2024-03-19,149.149,0,0,{},858.67,-7.0,851.67,-14306.0,-14206.0,1,-419.33,-13886.67
8,closed,USDJPYm,sell,100000,2024-03-22,151.626,2024-06-05,155.113,0,0,{},-2324.67,-7.0,-2331.67,-16637.67,-16537.67,2,-419.33,-16218.34
9,closed,USDJPYm,sell,100000,2024-06-21,158.931,2024-07-12,158.279,0,0,{},434.67,-7.0,427.67,-16210.0,-16110.0,4,-419.33,-15790.67


ValueError: Plotly Express cannot process wide-form data with columns of different type.